In [5]:
import sys, os
sys.path.insert(0, os.path.abspath("..")) 

import polars as pl
import numpy as np
from datetime import datetime

from graphglue.core.graph import Graph
from graphglue.adapters.sbml_adapter import from_sbml, BOUNDARY_SOURCE, BOUNDARY_SINK

# SBML adapter (Elowitz repressilator)

In [8]:
G = from_sbml("Elowitz.sbml.xml", graph=Graph(directed=True), preserve_stoichiometry=True)

print("vertices:", G.num_vertices)      # expect 8 (6 real + 2 boundary)
print("edges:", G.num_edges)            # expect 12
print("boundary nodes:", BOUNDARY_SOURCE in G.entity_to_idx, BOUNDARY_SINK in G.entity_to_idx)
print("sample edges:", list(G.edge_to_idx)[:5])


Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction1 "degradation of LacI transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction1 "degradation of LacI transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction2 "degradation of TetR transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction2 "degradation of TetR transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction3 "degradation of CI transcripts">'
Missing upper flux bound set to '1000.0' for reaction: '<Reaction Reaction3 "degradation of CI transcripts">'
Missing lower flux bound set to '-1000.0' for reaction: '<Reaction Reaction4 "translation of LacI">'
Missing upper flux bound se

vertices: 8
edges: 12
boundary nodes: True True
sample edges: ['Reaction1', 'Reaction2', 'Reaction3', 'Reaction4', 'Reaction5']


In [10]:
def show_reaction(eid: str):
    if eid not in G.edge_to_idx:
        print("no such edge:", eid); return
    h = G.hyperedge_definitions[eid]
    attrs = G.get_edge_attrs(eid)
    sto = attrs.get("stoich")  # present if you didn't add set_hyperedge_coeffs OR adapter stored it anyway
    print(f"[{eid}]")
    print("  head (products):", sorted(h["head"]))
    print("  tail (reactants):", sorted(h["tail"]))
    if sto:
        # filter zeros if any
        sto = {k: float(v) for k, v in sto.items() if abs(float(v)) > 1e-12}
        print("  stoich map:", sto)

# examples
show_reaction("Reaction1")
show_reaction("Reaction4")


[Reaction1]
  head (products): ['__BOUNDARY_SINK__']
  tail (reactants): ['X']
[Reaction4]
  head (products): ['PX']
  tail (reactants): ['__BOUNDARY_SOURCE__']


In [12]:
BOUNDARY = {BOUNDARY_SOURCE, BOUNDARY_SINK}

produced = {v:0 for v in G.entity_to_idx}
consumed = {v:0 for v in G.entity_to_idx}

for eid in G.edge_to_idx:
    h = G.hyperedge_definitions[eid]
    for v in h["head"]: produced[v] += 1
    for v in h["tail"]: consumed[v] += 1

real_species = [v for v in G.entity_to_idx if v not in BOUNDARY]
stats = [(v, produced[v], consumed[v]) for v in real_species]
stats.sort(key=lambda t: (t[1], t[2]), reverse=True)
for v,p,c in stats:
    print(f"{v:>4}  produced_in={p}  consumed_in={c}")


  PX  produced_in=1  consumed_in=1
  PY  produced_in=1  consumed_in=1
  PZ  produced_in=1  consumed_in=1
   X  produced_in=1  consumed_in=1
   Y  produced_in=1  consumed_in=1
   Z  produced_in=1  consumed_in=1


In [14]:
species_to_reactions = {v: {"as_product": [], "as_reactant": []} for v in G.entity_to_idx}

for eid in G.edge_to_idx:
    h = G.hyperedge_definitions[eid]
    for v in h["head"]:
        species_to_reactions[v]["as_product"].append(eid)
    for v in h["tail"]:
        species_to_reactions[v]["as_reactant"].append(eid)

# example: show for each real species
for v in real_species:
    rp = species_to_reactions[v]["as_product"]
    rr = species_to_reactions[v]["as_reactant"]
    print(f"\n{v}")
    print("  as product :", rp)
    print("  as reactant:", rr)



PX
  as product : ['Reaction4']
  as reactant: ['Reaction7']

PY
  as product : ['Reaction5']
  as reactant: ['Reaction8']

PZ
  as product : ['Reaction6']
  as reactant: ['Reaction9']

X
  as product : ['Reaction10']
  as reactant: ['Reaction1']

Y
  as product : ['Reaction11']
  as reactant: ['Reaction2']

Z
  as product : ['Reaction12']
  as reactant: ['Reaction3']


In [16]:
# signs: by definition head=products (+), tail=reactants (−)
def signs_consistent(eid):
    h = G.hyperedge_definitions[eid]
    attrs = G.get_edge_attrs(eid)
    sto = attrs.get("stoich")
    if not sto:
        # no per-vertex coeffs exposed via attrs; just check sets are present
        return bool(h["head"] or h["tail"])
    # if stoich map exists, check sign consistency vs head/tail sets
    ok = True
    for v, coeff in sto.items():
        coeff = float(coeff)
        if v in h["head"]: ok &= coeff > 0
        if v in h["tail"]: ok &= coeff < 0
    return ok

sign_ok_all = all(signs_consistent(e) for e in G.edge_to_idx)
print("signs consistent (based on available attrs):", sign_ok_all)

# balance: if stoich map exists, sum should be ~0 including boundary nodes
def balanced(eid):
    attrs = G.get_edge_attrs(eid)
    sto = attrs.get("stoich")
    if not sto: return True  # can't check without exposed coeffs; treat as pass
    s = sum(float(v) for v in sto.values())
    return abs(s) < 1e-9

bal_ok_all = all(balanced(e) for e in G.edge_to_idx)
print("columns balanced (based on available attrs):", bal_ok_all)


signs consistent (based on available attrs): True
columns balanced (based on available attrs): True


In [18]:
# degrees on your underlying NX projection (pass G explicitly)
deg = dict(G.nx.degree(G=G))
print("nx nodes:", G.nx.number_of_nodes(G=G), " nx edges:", G.nx.number_of_edges(G=G))
print("top-degree nodes:", sorted(deg.items(), key=lambda kv: kv[1], reverse=True)[:10])

# simple paths (between two species if connected in your projection)
try:
    path = G.nx.shortest_path(G=G, source="X", target="PX")  # tweak names if different
    print("shortest path X→PX:", path)
except Exception as e:
    print("shortest_path failed:", e)

# cycles (directed projection)
try:
    cyc = list(G.nx.simple_cycles(G=G))
    print("cycles(count):", len(cyc))
    print("sample cycles:", cyc[:3])
except Exception as e:
    print("simple_cycles failed:", e)

# neighbors / predecessors / successors
try:
    print("neighbors(X):", list(G.nx.neighbors(G=G, n="X")))
except Exception as e:
    print("neighbors failed:", e)

try:
    print("successors(X):", list(G.nx.successors(G=G, n="X")))
    print("predecessors(X):", list(G.nx.predecessors(G=G, n="X")))
except Exception:
    pass

# connected components (weakly for directed; else undirected)
try:
    comps = list(G.nx.weakly_connected_components(G=G))
    print("weakly components:", len(comps))
    print("largest component size:", max(len(c) for c in comps))
except Exception:
    try:
        comps = list(G.nx.connected_components(G=G))
        print("connected components:", len(comps))
        print("largest component size:", max(len(c) for c in comps))
    except Exception as e:
        print("components failed:", e)

# degree centrality (works the same way)
try:
    dc = G.nx.degree_centrality(G=G)
    print("top degree_centrality:", sorted(dc.items(), key=lambda kv: kv[1], reverse=True)[:5])
except Exception as e:
    print("degree_centrality failed:", e)

# species-only subgraph with the proxy (filters out boundary nodes)
BOUNDARY = {"__BOUNDARY_SOURCE__", "__BOUNDARY_SINK__"}
try:
    species = [n for n in G.nx.nodes(G=G) if n not in BOUNDARY]
    SG = G.nx.subgraph(G=G, nbunch=species)  # returns an NX graph
    print("species-subgraph nodes:", SG.number_of_nodes(), "edges:", SG.number_of_edges())
except Exception as e:
    print("subgraph failed:", e)


nx nodes: 8  nx edges: 12
top-degree nodes: [('__BOUNDARY_SOURCE__', 6), ('__BOUNDARY_SINK__', 6), ('PX', 2), ('PY', 2), ('PZ', 2), ('X', 2), ('Y', 2), ('Z', 2)]
shortest_path failed: No path between X and PX.
cycles(count): 0
sample cycles: []
neighbors(X): ['__BOUNDARY_SOURCE__']
weakly components: 1
largest component size: 8
top degree_centrality: [('__BOUNDARY_SOURCE__', 0.8571428571428571), ('__BOUNDARY_SINK__', 0.8571428571428571), ('PX', 0.2857142857142857), ('PY', 0.2857142857142857), ('PZ', 0.2857142857142857)]
species-subgraph nodes: 6 edges: 0


# AnnNet API

In [21]:
# Create graph
G = Graph(directed=True)

print("=" * 60)
print("CREATING MULTI-LAYER TEMPORAL GRAPH")
print("=" * 60)

# Add layers
G.layers.add("2022", year=2022, description="Year 2022")
G.layers.add("2023", year=2023, description="Year 2023")
G.layers.add("2024", year=2024, description="Year 2024")

print(f"\n✓ Created {G.layers.count()} layers")
print(f"  Layers: {G.layers.list()}")

CREATING MULTI-LAYER TEMPORAL GRAPH

✓ Created 4 layers
  Layers: ['2022', '2023', '2024']


In [23]:
print("\n" + "=" * 60)
print("LAYER 2022: Adding nodes")
print("=" * 60)

G.layers.active = "2022"

# Add people
people_2022 = {
    "alice": {"name": "Alice", "age": 25, "role": "engineer", "salary": 80000},
    "bob": {"name": "Bob", "age": 30, "role": "manager", "salary": 95000},
    "charlie": {"name": "Charlie", "age": 28, "role": "engineer", "salary": 85000},
    "diana": {"name": "Diana", "age": 35, "role": "director", "salary": 120000},
}

for vid, attrs in people_2022.items():
    G.add_vertex(vid, **attrs)

# Add collaborations (edges)
collaborations_2022 = [
    ("alice", "bob", 0.8, {"project": "ProjectX", "hours": 120}),
    ("alice", "charlie", 0.9, {"project": "ProjectX", "hours": 150}),
    ("bob", "diana", 0.7, {"project": "Management", "hours": 80}),
    ("charlie", "diana", 0.6, {"project": "ProjectY", "hours": 60}),
]

for source, target, weight, attrs in collaborations_2022:
    G.add_edge(source, target, weight=weight, **attrs)

print(f"\n✓ Layer 2022:")
print(f"  Vertices: {G.number_of_vertices()}")
print(f"  Edges: {G.number_of_edges()}")


LAYER 2022: Adding nodes

✓ Layer 2022:
  Vertices: 4
  Edges: 4


In [25]:
print("\n" + "=" * 60)
print("LAYER 2023: Adding nodes and edges")
print("=" * 60)

G.layers.active = "2023"

# Add existing people (some with updated attributes)
people_2023 = {
    "alice": {"name": "Alice", "age": 26, "role": "senior_engineer", "salary": 92000},
    "bob": {"name": "Bob", "age": 31, "role": "senior_manager", "salary": 105000},
    "charlie": {"name": "Charlie", "age": 29, "role": "engineer", "salary": 88000},
    "diana": {"name": "Diana", "age": 36, "role": "director", "salary": 125000},
    "eve": {"name": "Eve", "age": 27, "role": "engineer", "salary": 83000},  # New hire
}

for vid, attrs in people_2023.items():
    if not G.has_vertex(vid):
        G.add_vertex(vid, **attrs)

# New collaborations
collaborations_2023 = [
    ("alice", "bob", 0.85, {"project": "ProjectZ", "hours": 140}),
    ("alice", "eve", 0.95, {"project": "ProjectZ", "hours": 180}),  # New collaboration
    ("bob", "diana", 0.75, {"project": "Management", "hours": 90}),
    ("charlie", "eve", 0.8, {"project": "ProjectW", "hours": 100}),
    ("eve", "diana", 0.7, {"project": "ProjectW", "hours": 70}),
]

for source, target, weight, attrs in collaborations_2023:
    G.add_edge(source, target, weight=weight, **attrs)

print(f"\n✓ Layer 2023:")
print(f"  Total vertices: {G.number_of_vertices()}")
print(f"  Edges in layer: {len(G.layers.edges('2023'))}")


LAYER 2023: Adding nodes and edges

✓ Layer 2023:
  Total vertices: 5
  Edges in layer: 5


In [27]:
print("\n" + "=" * 60)
print("LAYER 2024: Adding nodes and edges")
print("=" * 60)

G.layers.active = "2024"

# 2024 people (Bob left, Frank joined)
people_2024 = {
    "alice": {"name": "Alice", "age": 27, "role": "tech_lead", "salary": 110000},
    "charlie": {"name": "Charlie", "age": 30, "role": "senior_engineer", "salary": 98000},
    "diana": {"name": "Diana", "age": 37, "role": "vp", "salary": 150000},
    "eve": {"name": "Eve", "age": 28, "role": "senior_engineer", "salary": 95000},
    "frank": {"name": "Frank", "age": 32, "role": "manager", "salary": 100000},  # Replaced Bob
}

for vid, attrs in people_2024.items():
    if not G.has_vertex(vid):
        G.add_vertex(vid, **attrs)

# 2024 collaborations
collaborations_2024 = [
    ("alice", "frank", 0.9, {"project": "NextGen", "hours": 160}),
    ("alice", "eve", 0.92, {"project": "NextGen", "hours": 170}),
    ("charlie", "eve", 0.85, {"project": "NextGen", "hours": 120}),
    ("frank", "diana", 0.8, {"project": "Strategy", "hours": 100}),
    ("eve", "diana", 0.75, {"project": "Strategy", "hours": 80}),
]

for source, target, weight, attrs in collaborations_2024:
    G.add_edge(source, target, weight=weight, **attrs)

print(f"\n✓ Layer 2024:")
print(f"  Total vertices: {G.number_of_vertices()}")
print(f"  Edges in layer: {len(G.layers.edges('2024'))}")

print(f"\n{'='*60}")
print("GRAPH SUMMARY")
print(f"{'='*60}")
print(f"Total unique vertices: {G.number_of_vertices()}")
print(f"Total unique edges: {G.number_of_edges()}")
print(f"Layers: {G.layers.count()}")


LAYER 2024: Adding nodes and edges

✓ Layer 2024:
  Total vertices: 6
  Edges in layer: 5

GRAPH SUMMARY
Total unique vertices: 6
Total unique edges: 14
Layers: 4


In [29]:
print("\n" + "=" * 60)
print("TESTING ANNNET PROPERTIES")
print("=" * 60)

# Test obs (vertex attributes)
print("\n1. obs (vertex attributes):")
print(G.obs)
print(f"\n   Shape: {G.obs.shape}")
print(f"   Columns: {G.obs.columns}")

# Test var (edge attributes)
print("\n2. var (edge attributes):")
print(G.var.head())
print(f"\n   Shape: {G.var.shape}")
print(f"   Columns: {G.var.columns}")

# Test X (incidence matrix)
print("\n3. X (incidence matrix):")
X = G.X()
print(f"   Type: {type(X)}")
print(f"   Shape: {X.shape}")
print(f"   Non-zero entries: {X.nnz}")
print(f"   Density: {X.nnz / (X.shape[0] * X.shape[1]):.4f}")

# Test uns (unstructured metadata)
print("\n4. uns (unstructured metadata):")
G.uns["dataset_name"] = "Company Collaboration Network"
G.uns["created"] = datetime.now().isoformat()
G.uns["description"] = "Multi-year collaboration network"
print(f"   {G.uns}")


TESTING ANNNET PROPERTIES

1. obs (vertex attributes):
shape: (6, 5)
┌───────────┬─────────┬─────┬──────────┬────────┐
│ vertex_id ┆ name    ┆ age ┆ role     ┆ salary │
│ ---       ┆ ---     ┆ --- ┆ ---      ┆ ---    │
│ str       ┆ str     ┆ i64 ┆ str      ┆ i64    │
╞═══════════╪═════════╪═════╪══════════╪════════╡
│ alice     ┆ Alice   ┆ 25  ┆ engineer ┆ 80000  │
│ bob       ┆ Bob     ┆ 30  ┆ manager  ┆ 95000  │
│ charlie   ┆ Charlie ┆ 28  ┆ engineer ┆ 85000  │
│ diana     ┆ Diana   ┆ 35  ┆ director ┆ 120000 │
│ eve       ┆ Eve     ┆ 27  ┆ engineer ┆ 83000  │
│ frank     ┆ Frank   ┆ 32  ┆ manager  ┆ 100000 │
└───────────┴─────────┴─────┴──────────┴────────┘

   Shape: (6, 5)
   Columns: ['vertex_id', 'name', 'age', 'role', 'salary']

2. var (edge attributes):
shape: (5, 3)
┌─────────┬────────────┬───────┐
│ edge_id ┆ project    ┆ hours │
│ ---     ┆ ---        ┆ ---   │
│ str     ┆ str        ┆ i64   │
╞═════════╪════════════╪═══════╡
│ edge_0  ┆ ProjectX   ┆ 120   │
│ edge_1  ┆ Pr

In [31]:
print("\n" + "=" * 60)
print("TESTING LAYERMANAGER")
print("=" * 60)

# Basic operations
print("\n1. Layer Info:")
print(f"   Active layer: {G.layers.active}")
print(f"   All layers: {G.layers.list()}")
print(f"   Layer count: {G.layers.count()}")

# Layer statistics
print("\n2. Layer Statistics:")
stats = G.layers.stats()
for layer_id, info in stats.items():
    print(f"\n   {layer_id}:")
    print(f"     Vertices: {info['vertices']}")
    print(f"     Edges: {info['edges']}")
    print(f"     Attributes: {info['attributes']}")

# Layer operations - union
print("\n3. Union of 2022 and 2023:")
union_result = G.layers.union(["2022", "2023"])
print(f"   Vertices: {len(union_result['vertices'])}")
print(f"   Edges: {len(union_result['edges'])}")
print(f"   Vertex IDs: {sorted(union_result['vertices'])}")

# Layer operations - intersection
print("\n4. Intersection of 2022 and 2023:")
intersect_result = G.layers.intersect(["2022", "2023"])
print(f"   Common vertices: {sorted(intersect_result['vertices'])}")
print(f"   Common edges: {len(intersect_result['edges'])}")

# Create aggregated layer
print("\n5. Create 'all_years' layer (union):")
G.layers.union_create(["2022", "2023", "2024"], "all_years", 
                     description="All years combined")
print(f"   ✓ Created layer: all_years")
print(f"   Vertices: {len(G.layers.vertices('all_years'))}")
print(f"   Edges: {len(G.layers.edges('all_years'))}")

# Summary
print("\n6. Layer Summary:")
print(G.layers.summary())


TESTING LAYERMANAGER

1. Layer Info:
   Active layer: 2024
   All layers: ['2022', '2023', '2024']
   Layer count: 4

2. Layer Statistics:

   2022:
     Vertices: 4
     Edges: 4
     Attributes: {'year': 2022, 'description': 'Year 2022'}

   2023:
     Vertices: 5
     Edges: 5
     Attributes: {'year': 2023, 'description': 'Year 2023'}

   2024:
     Vertices: 5
     Edges: 5
     Attributes: {'year': 2024, 'description': 'Year 2024'}

3. Union of 2022 and 2023:
   Vertices: 5
   Edges: 9
   Vertex IDs: ['alice', 'bob', 'charlie', 'diana', 'eve']

4. Intersection of 2022 and 2023:
   Common vertices: ['alice', 'bob', 'charlie', 'diana']
   Common edges: 0

5. Create 'all_years' layer (union):
   ✓ Created layer: all_years
   Vertices: 6
   Edges: 14

6. Layer Summary:
Layers: 5
├─ default: 0 vertices, 0 edges
├─ 2022: 4 vertices, 4 edges
├─ 2023: 5 vertices, 5 edges
├─ 2024: 5 vertices, 5 edges
└─ all_years: 6 vertices, 14 edges


In [33]:
print("\n" + "=" * 60)
print("TESTING CROSS-LAYER ANALYTICS")
print("=" * 60)

# Vertex presence
print("\n1. Vertex Presence Across Layers:")
for vid in ["alice", "bob", "eve", "frank"]:
    layers = G.layers.vertex_presence(vid)
    print(f"   {vid}: {layers}")

# Edge presence
print("\n2. Edge Presence (alice→bob):")
edge_presence = G.layers.edge_presence(source="alice", target="bob")
for layer_id, edge_ids in edge_presence.items():
    print(f"   {layer_id}: {edge_ids}")

# Conserved edges
print("\n3. Conserved Edges (in 2+ layers):")
conserved = G.layers.conserved_edges(min_layers=2)
print(f"   Found {len(conserved)} conserved edges:")
for eid, count in sorted(conserved.items(), key=lambda x: x[1], reverse=True)[:5]:
    edge_def = G.edge_definitions.get(eid)
    if edge_def:
        print(f"   {eid}: {edge_def[0]} → {edge_def[1]} (in {count} layers)")

# Layer-specific edges
print("\n4. Layer-Specific Edges:")
for layer_id in ["2022", "2023", "2024"]:
    specific = G.layers.specific_edges(layer_id)
    print(f"   {layer_id} only: {len(specific)} edges")

# Temporal dynamics
print("\n5. Temporal Dynamics:")
changes = G.layers.temporal_dynamics(["2022", "2023", "2024"], metric="edge_change")
for i, change in enumerate(changes):
    year_from = ["2022", "2023"][i]
    year_to = ["2023", "2024"][i]
    print(f"\n   {year_from} → {year_to}:")
    print(f"     Edges added: {change['added']}")
    print(f"     Edges removed: {change['removed']}")
    print(f"     Net change: {change['net_change']:+d}")


TESTING CROSS-LAYER ANALYTICS

1. Vertex Presence Across Layers:
   alice: ['2022', '2023', '2024', 'all_years']
   bob: ['2022', '2023', 'all_years']
   eve: ['2023', '2024', 'all_years']
   frank: ['2024', 'all_years']

2. Edge Presence (alice→bob):
   2022: ['edge_0']
   2023: ['edge_4']
   all_years: ['edge_4', 'edge_0']

3. Conserved Edges (in 2+ layers):
   Found 14 conserved edges:
   edge_3: charlie → diana (in 2 layers)
   edge_1: alice → charlie (in 2 layers)
   edge_0: alice → bob (in 2 layers)
   edge_2: bob → diana (in 2 layers)
   edge_4: alice → bob (in 2 layers)

4. Layer-Specific Edges:
   2022 only: 0 edges
   2023 only: 0 edges
   2024 only: 0 edges

5. Temporal Dynamics:

   2022 → 2023:
     Edges added: 5
     Edges removed: 4
     Net change: +1

   2023 → 2024:
     Edges added: 5
     Edges removed: 5
     Net change: +0


In [35]:
print("\n" + "=" * 60)
print("TESTING INDEXMANAGER")
print("=" * 60)

# Entity lookups
print("\n1. Entity Index Lookups:")
print(f"   alice → row index: {G.idx.entity_to_row('alice')}")
print(f"   diana → row index: {G.idx.entity_to_row('diana')}")
print(f"   Row 0 → entity: {G.idx.row_to_entity(0)}")
print(f"   Row 3 → entity: {G.idx.row_to_entity(3)}")

# Edge lookups
print("\n2. Edge Index Lookups:")
edge_ids = list(G.edge_to_idx.keys())[:3]
for eid in edge_ids:
    col = G.idx.edge_to_col(eid)
    back = G.idx.col_to_edge(col)
    print(f"   {eid} → col {col} → {back}")

# Batch lookups
print("\n3. Batch Lookups:")
vertices = ["alice", "bob", "charlie"]
rows = G.idx.entities_to_rows(vertices)
print(f"   {vertices}")
print(f"   → rows: {rows}")
back_entities = G.idx.rows_to_entities(rows)
print(f"   → back: {back_entities}")

# Check existence
print("\n4. Existence Checks:")
print(f"   'alice' exists: {G.idx.has_entity('alice')}")
print(f"   'unknown' exists: {G.idx.has_entity('unknown')}")
print(f"   edge count: {G.idx.edge_count()}")
print(f"   entity count: {G.idx.entity_count()}")


TESTING INDEXMANAGER

1. Entity Index Lookups:
   alice → row index: 0
   diana → row index: 3
   Row 0 → entity: alice
   Row 3 → entity: diana

2. Edge Index Lookups:
   edge_0 → col 0 → edge_0
   edge_1 → col 1 → edge_1
   edge_2 → col 2 → edge_2

3. Batch Lookups:
   ['alice', 'bob', 'charlie']
   → rows: [0, 1, 2]
   → back: ['alice', 'bob', 'charlie']

4. Existence Checks:
   'alice' exists: True
   'unknown' exists: False
   edge count: 14
   entity count: 6


In [37]:
print("\n" + "=" * 60)
print("TESTING CACHEMANAGER")
print("=" * 60)

# Check cache status
print("\n1. Initial Cache Status:")
print(f"   CSR cached: {G.cache.has_csr()}")
print(f"   CSC cached: {G.cache.has_csc()}")

# Build CSR
print("\n2. Building CSR cache...")
import time
t0 = time.time()
csr = G.cache.get_csr()
t1 = time.time()
print(f"   ✓ Built in {(t1-t0)*1000:.2f}ms")
print(f"   Shape: {csr.shape}")
print(f"   Type: {type(csr)}")

# Build CSC
print("\n3. Building CSC cache...")
t0 = time.time()
csc = G.cache.get_csc()
t1 = time.time()
print(f"   ✓ Built in {(t1-t0)*1000:.2f}ms")
print(f"   Shape: {csc.shape}")

# Check cache hit
print("\n4. Cache Hit Test:")
t0 = time.time()
csr2 = G.cache.get_csr()  # Should be instant
t1 = time.time()
print(f"   ✓ Retrieved in {(t1-t0)*1000:.4f}ms (cached)")

# Clear cache
print("\n5. Clearing Cache:")
G.cache.clear()
print(f"   CSR cached: {G.cache.has_csr()}")
print(f"   CSC cached: {G.cache.has_csc()}")

# Rebuild
print("\n6. Rebuild All:")
G.cache.build()
print(f"   ✓ CSR cached: {G.cache.has_csr()}")
print(f"   ✓ CSC cached: {G.cache.has_csc()}")


TESTING CACHEMANAGER

1. Initial Cache Status:
   CSR cached: False
   CSC cached: False

2. Building CSR cache...
   ✓ Built in 0.93ms
   Shape: (8, 16)
   Type: <class 'scipy.sparse._csr.csr_matrix'>

3. Building CSC cache...
   ✓ Built in 1.08ms
   Shape: (8, 16)

4. Cache Hit Test:
   ✓ Retrieved in 0.0000ms (cached)

5. Clearing Cache:
   CSR cached: False
   CSC cached: False

6. Rebuild All:
   ✓ CSR cached: True
   ✓ CSC cached: True


In [39]:
print("\n" + "=" * 60)
print("TESTING GRAPHVIEW - BASIC FILTERING")
print("=" * 60)

# View specific nodes
print("\n1. View Specific Nodes:")
v = G.view(nodes=["alice", "bob", "charlie"])
print(f"   View: {v}")
print(f"   Nodes: {v.node_count}")
print(f"   Edges: {v.edge_count}")
print(f"\n   Node table:")
print(v.obs)

# View specific layer
print("\n2. View Layer 2023:")
v2023 = G.view(layers="2023")
print(f"   View: {v2023}")
print(f"   Nodes: {v2023.node_count}")
print(f"   Edges: {v2023.edge_count}")

# View multiple layers
print("\n3. View Layers 2022+2023:")
v_early = G.view(layers=["2022", "2023"])
print(f"   View: {v_early}")
print(f"   Nodes: {v_early.node_count}")
print(f"   Edges: {v_early.edge_count}")

# Combined filters
print("\n4. Combined: Specific nodes in 2023:")
v_combo = G.view(nodes=["alice", "eve"], layers="2023")
print(f"   View: {v_combo}")
print(f"   Nodes: {v_combo.node_count}")
print(f"   Edges: {v_combo.edge_count}")


TESTING GRAPHVIEW - BASIC FILTERING

1. View Specific Nodes:
   View: GraphView(nodes=3, edges=14)
   Nodes: 3
   Edges: 14

   Node table:
shape: (3, 5)
┌───────────┬─────────┬─────┬──────────┬────────┐
│ vertex_id ┆ name    ┆ age ┆ role     ┆ salary │
│ ---       ┆ ---     ┆ --- ┆ ---      ┆ ---    │
│ str       ┆ str     ┆ i64 ┆ str      ┆ i64    │
╞═══════════╪═════════╪═════╪══════════╪════════╡
│ alice     ┆ Alice   ┆ 25  ┆ engineer ┆ 80000  │
│ bob       ┆ Bob     ┆ 30  ┆ manager  ┆ 95000  │
│ charlie   ┆ Charlie ┆ 28  ┆ engineer ┆ 85000  │
└───────────┴─────────┴─────┴──────────┴────────┘

2. View Layer 2023:
   View: GraphView(nodes=5, edges=5)
   Nodes: 5
   Edges: 5

3. View Layers 2022+2023:
   View: GraphView(nodes=5, edges=9)
   Nodes: 5
   Edges: 9

4. Combined: Specific nodes in 2023:
   View: GraphView(nodes=2, edges=1)
   Nodes: 2
   Edges: 1


In [41]:
print("\n" + "=" * 60)
print("TESTING GRAPHVIEW - PREDICATE FILTERING")
print("=" * 60)

# View by node predicate (high salary)
print("\n1. High Salary Employees (>100k):")
v_rich = G.view(nodes=lambda vid: 
    G.get_vertex_attrs(vid).get("salary", 0) > 100000
)
print(f"   View: {v_rich}")
print(f"   High earners: {sorted(v_rich.node_ids)}")
print(f"\n   Details:")
print(v_rich.obs.select(["vertex_id", "name", "salary", "role"]))

# View by edge predicate (strong collaboration)
print("\n2. Strong Collaborations (weight > 0.8):")
v_strong = G.view(edges=lambda eid:
    G.edge_weights.get(eid, 0) > 0.8
)
print(f"   View: {v_strong}")
print(f"   Strong edges: {v_strong.edge_count}")
edges_df = v_strong.edges_df(include_weight=True)
print(f"\n   Top collaborations:")
print(edges_df.select(["edge_id", "source", "target", "global_weight"]).head(10))

# Combined predicate (engineers in recent years)
print("\n3. Engineers in 2023/2024:")
v_eng = G.view(
    nodes=lambda vid: "engineer" in G.get_vertex_attrs(vid).get("role", ""),
    layers=["2023", "2024"]
)
print(f"   View: {v_eng}")
print(f"   Engineers: {sorted(v_eng.node_ids)}")


TESTING GRAPHVIEW - PREDICATE FILTERING

1. High Salary Employees (>100k):
   View: GraphView(nodes=1, edges=14)
   High earners: ['diana']

   Details:
shape: (1, 4)
┌───────────┬───────┬────────┬──────────┐
│ vertex_id ┆ name  ┆ salary ┆ role     │
│ ---       ┆ ---   ┆ ---    ┆ ---      │
│ str       ┆ str   ┆ i64    ┆ str      │
╞═══════════╪═══════╪════════╪══════════╡
│ diana     ┆ Diana ┆ 120000 ┆ director │
└───────────┴───────┴────────┴──────────┘

2. Strong Collaborations (weight > 0.8):
   View: GraphView(nodes=6, edges=6)
   Strong edges: 6

   Top collaborations:
shape: (6, 4)
┌─────────┬─────────┬─────────┬───────────────┐
│ edge_id ┆ source  ┆ target  ┆ global_weight │
│ ---     ┆ ---     ┆ ---     ┆ ---           │
│ str     ┆ str     ┆ str     ┆ f64           │
╞═════════╪═════════╪═════════╪═══════════════╡
│ edge_1  ┆ alice   ┆ charlie ┆ 0.9           │
│ edge_4  ┆ alice   ┆ bob     ┆ 0.85          │
│ edge_5  ┆ alice   ┆ eve     ┆ 0.95          │
│ edge_9  ┆ alice 

In [43]:
print("\n" + "=" * 60)
print("TESTING GRAPHVIEW - ADVANCED OPERATIONS")
print("=" * 60)

# Access properties
print("\n1. View Properties:")
v = G.view(layers="2023")
print(f"   node_count: {v.node_count}")
print(f"   edge_count: {v.edge_count}")
print(f"   node_ids: {sorted(list(v.node_ids))}")
print(f"\n   Matrix shape: {v.X.shape}")
print(f"   Matrix nnz: {v.X.nnz}")

# Get DataFrames
print("\n2. View DataFrames:")
vertices_df = v.vertices_df()
edges_df = v.edges_df(include_weight=True, include_directed=True)
print(f"   Vertices DF: {vertices_df.shape}")
print(f"   Edges DF: {edges_df.shape}")

# Summary
print("\n3. View Summary:")
print(v.summary())

# Nested views
print("\n4. Nested Views:")
v1 = G.view(layers="2023")
print(f"   v1 (2023): {v1.node_count} nodes, {v1.edge_count} edges")

v2 = v1.subview(nodes=["alice", "bob", "eve"])
print(f"   v2 (alice/bob/eve in 2023): {v2.node_count} nodes, {v2.edge_count} edges")


TESTING GRAPHVIEW - ADVANCED OPERATIONS

1. View Properties:
   node_count: 5
   edge_count: 5
   node_ids: ['alice', 'bob', 'charlie', 'diana', 'eve']

   Matrix shape: (5, 5)
   Matrix nnz: 10

2. View DataFrames:
   Vertices DF: (5, 5)
   Edges DF: (5, 13)

3. View Summary:
GraphView Summary
──────────────────────────────
Nodes: 5
Edges: 5
Filters: layers=['2023']

4. Nested Views:
   v1 (2023): 5 nodes, 5 edges
   v2 (alice/bob/eve in 2023): 3 nodes, 2 edges


In [45]:
print("\n" + "=" * 60)
print("TESTING GRAPHVIEW - MATERIALIZATION")
print("=" * 60)

# Materialize layer 2023
print("\n1. Materialize Layer 2023:")
v2023 = G.view(layers="2023")
subG = v2023.materialize(copy_attributes=True)

print(f"   Original graph: {G.number_of_vertices()} nodes, {G.number_of_edges()} edges")
print(f"   Subgraph 2023: {subG.number_of_vertices()} nodes, {subG.number_of_edges()} edges")
print(f"   Subgraph vertices: {sorted(subG.vertices())}")

# Check attributes were copied
print(f"\n   Sample attributes:")
alice_attrs = subG.get_vertex_attrs("alice")
print(f"   alice: {alice_attrs}")

# Materialize high earners
print("\n2. Materialize High Earners Network:")
v_rich = G.view(nodes=lambda vid: 
    G.get_vertex_attrs(vid).get("salary", 0) > 95000
)
rich_network = v_rich.materialize(copy_attributes=True)

print(f"   High earners network: {rich_network.number_of_vertices()} nodes")
print(f"   Nodes: {sorted(rich_network.vertices())}")
print(f"   Edges: {rich_network.number_of_edges()}")

# Verify independence
print("\n3. Verify Independence:")
print(f"   Original graph edges: {G.number_of_edges()}")
print(f"   Subgraph edges: {subG.number_of_edges()}")
subG.add_vertex("test_node")
print(f"   After modifying subgraph:")
print(f"     Original: {G.number_of_vertices()} nodes")
print(f"     Subgraph: {subG.number_of_vertices()} nodes")
print(f"   ✓ Graphs are independent")


TESTING GRAPHVIEW - MATERIALIZATION

1. Materialize Layer 2023:
   Original graph: 6 nodes, 14 edges
   Subgraph 2023: 5 nodes, 5 edges
   Subgraph vertices: ['alice', 'bob', 'charlie', 'diana', 'eve']

   Sample attributes:
   alice: {'vertex_id': 'alice', 'name': 'Alice', 'age': 25, 'role': 'engineer', 'salary': 80000}

2. Materialize High Earners Network:
   High earners network: 2 nodes
   Nodes: ['diana', 'frank']
   Edges: 1

3. Verify Independence:
   Original graph edges: 14
   Subgraph edges: 5
   After modifying subgraph:
     Original: 6 nodes
     Subgraph: 6 nodes
   ✓ Graphs are independent


In [47]:
print("\n" + "=" * 60)
print("TESTING SNAPSHOT AND DIFF")
print("=" * 60)

# Create initial snapshot
print("\n1. Create Initial Snapshot:")
G.layers.active = "2024"
snap1 = G.snapshot("initial_state")
print(f"   ✓ Created snapshot: {snap1['label']}")
print(f"   Vertices: {snap1['counts']['vertices']}")
print(f"   Edges: {snap1['counts']['edges']}")
print(f"   Layers: {snap1['counts']['layers']}")

# Make changes
print("\n2. Make Changes:")
print("   Adding new vertices...")
G.add_vertex("grace", name="Grace", age=29, role="engineer", salary=87000)
G.add_vertex("henry", name="Henry", age=33, role="architect", salary=115000)

print("   Adding new edges...")
G.add_edge("grace", "alice", weight=0.85, project="Innovation")
G.add_edge("henry", "diana", weight=0.9, project="Architecture")

print("   Removing a vertex...")
G.remove_vertex("frank")

# Create second snapshot
snap2 = G.snapshot("after_changes")
print(f"\n   ✓ Created snapshot: {snap2['label']}")

# Diff
print("\n3. Compare Snapshots:")
diff = G.diff("initial_state", "after_changes")
print(diff.summary())

print(f"\n   Details:")
print(f"   Added vertices: {sorted(diff.vertices_added)}")
print(f"   Removed vertices: {sorted(diff.vertices_removed)}")
print(f"   Added edges: {len(diff.edges_added)}")
print(f"   Removed edges: {len(diff.edges_removed)}")

# Compare with current
print("\n4. Compare with Current State:")
G.add_vertex("iris", name="Iris", age=26, role="data_scientist", salary=92000)
diff_current = G.diff("after_changes")
print(diff_current.summary())

# List all snapshots
print("\n5. List All Snapshots:")
snapshots = G.list_snapshots()
for snap in snapshots:
    print(f"\n   {snap['label']}:")
    print(f"     Timestamp: {snap['timestamp']}")
    print(f"     Vertices: {snap['counts']['vertices']}")
    print(f"     Edges: {snap['counts']['edges']}")


TESTING SNAPSHOT AND DIFF

1. Create Initial Snapshot:
   ✓ Created snapshot: initial_state
   Vertices: 6
   Edges: 14
   Layers: 5

2. Make Changes:
   Adding new vertices...
   Adding new edges...
   Removing a vertex...

   ✓ Created snapshot: after_changes

3. Compare Snapshots:
Diff: initial_state → after_changes

Vertices: +2 added, 1 removed
Edges: +2 added, 2 removed
Layers: +0 added, 0 removed

   Details:
   Added vertices: ['grace', 'henry']
   Removed vertices: ['frank']
   Added edges: 2
   Removed edges: 2

4. Compare with Current State:
Diff: after_changes → current

Vertices: +1 added, 0 removed
Edges: +0 added, 0 removed
Layers: +0 added, 0 removed

5. List All Snapshots:

   initial_state:
     Timestamp: 2025-10-23T19:01:31.548907+00:00
     Vertices: 6
     Edges: 14

   after_changes:
     Timestamp: 2025-10-23T19:01:31.553366+00:00
     Vertices: 7
     Edges: 14


In [56]:
print("\n" + "=" * 60)
print("ADVANCED ANALYSIS - NETWORK METRICS")
print("=" * 60)

# Per-layer analysis
print("\n1. Per-Layer Network Metrics:")
for layer_id in ["2022", "2023", "2024"]:
    v = G.view(layers=layer_id)
    
    print(f"\n   {layer_id}:")
    print(f"     Nodes: {v.node_count}")
    print(f"     Edges: {v.edge_count}")
    
    if v.edge_count > 0:
        avg_weight = v.var.select("weight").mean().item() if "weight" in v.var.columns else 0
        print(f"     Avg edge weight: {avg_weight:.3f}")
    
    # Degree analysis (using materialized subgraph)
    subG = v.materialize()
    degrees = {vid: subG.degree(vid) for vid in subG.vertices()}
    if degrees:
        print(f"     Avg degree: {sum(degrees.values()) / len(degrees):.2f}")
        print(f"     Max degree: {max(degrees.values())}")
        max_degree_node = max(degrees, key=degrees.get)
        print(f"     Hub: {max_degree_node} (degree={degrees[max_degree_node]})")

# Compare layers
print("\n2. Layer Comparison:")
changes = G.layers.temporal_dynamics(["2022", "2023", "2024"], metric="vertex_change")
print("\n   Vertex changes over time:")
for i, change in enumerate(changes):
    year_from = ["2022", "2023"][i]
    year_to = ["2023", "2024"][i]
    print(f"   {year_from}→{year_to}: {change['added']:+d} added, {change['removed']} removed")

changes = G.layers.temporal_dynamics(["2022", "2023", "2024"], metric="edge_change")
print("\n   Edge changes over time:")
for i, change in enumerate(changes):
    year_from = ["2022", "2023"][i]
    year_to = ["2023", "2024"][i]
    print(f"   {year_from}→{year_to}: {change['added']:+d} added, {change['removed']} removed")


ADVANCED ANALYSIS - NETWORK METRICS

1. Per-Layer Network Metrics:

   2022:
     Nodes: 4
     Edges: 4
     Avg edge weight: 0.000
     Avg degree: 2.00
     Max degree: 2
     Hub: charlie (degree=2)

   2023:
     Nodes: 5
     Edges: 5
     Avg edge weight: 0.000
     Avg degree: 2.00
     Max degree: 3
     Hub: eve (degree=3)

   2024:
     Nodes: 7
     Edges: 5
     Avg edge weight: 0.000
     Avg degree: 1.43
     Max degree: 3
     Hub: eve (degree=3)

2. Layer Comparison:

   Vertex changes over time:
   2022→2023: +1 added, 0 removed
   2023→2024: +3 added, 1 removed

   Edge changes over time:
   2022→2023: +5 added, 4 removed
   2023→2024: +5 added, 5 removed


In [58]:
print("\n" + "=" * 60)
print("ADVANCED ANALYSIS - POLARS QUERIES")
print("=" * 60)

# Query 1: Top earners
print("\n1. Top 3 Earners:")
top_earners = G.obs.sort("salary", descending=True).head(3)
print(top_earners.select(["vertex_id", "name", "role", "salary"]))

# Query 2: Role distribution
print("\n2. Role Distribution:")
role_dist = G.obs.group_by("role").agg([
    pl.count("vertex_id").alias("count"),
    pl.mean("salary").alias("avg_salary")
]).sort("count", descending=True)
print(role_dist)

# Query 3: High-weight collaborations
print("\n3. Top 5 Collaborations by Weight:")
top_edges = (
    G.view(layers="2023")
     .edges_df(layer="2023", include_weight=True, resolved_weight=True)  # adds global_weight, layer_weight, effective_weight
     .sort("effective_weight", descending=True)
     .select(["edge_id", "effective_weight"])
     .head(5)
)
print(top_edges)

# Query 4: Projects by hours
print("\n4. Total Hours by Project:")
if "project" in G.var.columns and "hours" in G.var.columns:
    project_hours = G.var.group_by("project").agg([
        pl.count("edge_id").alias("collaborations"),
        pl.sum("hours").alias("total_hours")
    ]).sort("total_hours", descending=True)
    print(project_hours)

# Query 5: Salary growth (across snapshots if attributes updated)
print("\n5. Salary Statistics:")
salary_stats = G.obs.select([
    pl.col("salary").min().alias("min_salary"),
    pl.col("salary").max().alias("max_salary"),
    pl.col("salary").mean().alias("avg_salary"),
    pl.col("salary").median().alias("median_salary")
])
print(salary_stats)


ADVANCED ANALYSIS - POLARS QUERIES

1. Top 3 Earners:
shape: (3, 4)
┌───────────┬───────┬───────────┬────────┐
│ vertex_id ┆ name  ┆ role      ┆ salary │
│ ---       ┆ ---   ┆ ---       ┆ ---    │
│ str       ┆ str   ┆ str       ┆ i64    │
╞═══════════╪═══════╪═══════════╪════════╡
│ diana     ┆ Diana ┆ director  ┆ 120000 │
│ henry     ┆ Henry ┆ architect ┆ 115000 │
│ bob       ┆ Bob   ┆ manager   ┆ 95000  │
└───────────┴───────┴───────────┴────────┘

2. Role Distribution:
shape: (5, 3)
┌────────────────┬───────┬────────────┐
│ role           ┆ count ┆ avg_salary │
│ ---            ┆ ---   ┆ ---        │
│ str            ┆ u32   ┆ f64        │
╞════════════════╪═══════╪════════════╡
│ engineer       ┆ 4     ┆ 83750.0    │
│ data_scientist ┆ 1     ┆ 92000.0    │
│ director       ┆ 1     ┆ 120000.0   │
│ manager        ┆ 1     ┆ 95000.0    │
│ architect      ┆ 1     ┆ 115000.0   │
└────────────────┴───────┴────────────┘

3. Top 5 Collaborations by Weight:
shape: (5, 2)
┌─────────┬──────

In [60]:
print("\n" + "=" * 60)
print("PERFORMANCE BENCHMARKS")
print("=" * 60)

import time

# Benchmark 1: View creation
print("\n1. View Creation (1000 iterations):")
t0 = time.time()
for _ in range(1000):
    v = G.view(layers="2023")
t1 = time.time()
print(f"   Time: {(t1-t0)*1000:.2f}ms total ({(t1-t0):.4f}ms per view)")

# Benchmark 2: Property access
print("\n2. Property Access (1000 iterations):")
v = G.view(layers="2023")
t0 = time.time()
for _ in range(1000):
    _ = v.obs
    _ = v.var
t1 = time.time()
print(f"   Time: {(t1-t0)*1000:.2f}ms total")

# Benchmark 3: Materialization
print("\n3. Materialization (100 iterations):")
v = G.view(layers="2023")
t0 = time.time()
for _ in range(100):
    subG = v.materialize(copy_attributes=False)
t1 = time.time()
print(f"   Time: {(t1-t0)*1000:.2f}ms total ({(t1-t0)*10:.2f}ms per materialization)")

# Benchmark 4: Snapshot creation
print("\n4. Snapshot Creation (100 iterations):")
t0 = time.time()
for i in range(100):
    G.snapshot(f"bench_{i}")
t1 = time.time()
print(f"   Time: {(t1-t0)*1000:.2f}ms total ({(t1-t0)*10:.2f}ms per snapshot)")
print(f"   Total snapshots: {len(G._snapshots)}")

# Benchmark 5: DataFrame filtering
print("\n5. DataFrame Filtering (1000 iterations):")
t0 = time.time()
for _ in range(1000):
    filtered = G.obs.filter(pl.col("salary") > 90000)
t1 = time.time()
print(f"   Time: {(t1-t0)*1000:.2f}ms total ({(t1-t0):.4f}ms per filter)")


PERFORMANCE BENCHMARKS

1. View Creation (1000 iterations):
   Time: 1.00ms total (0.0010ms per view)

2. Property Access (1000 iterations):
   Time: 523.89ms total

3. Materialization (100 iterations):
   Time: 89.99ms total (0.90ms per materialization)

4. Snapshot Creation (100 iterations):
   Time: 0.00ms total (0.00ms per snapshot)
   Total snapshots: 102

5. DataFrame Filtering (1000 iterations):
   Time: 194.86ms total (0.1949ms per filter)


In [62]:
print("\n" + "=" * 60)
print("ANNNET COMPLETE TEST SUMMARY")
print("=" * 60)

print("\n📊 GRAPH STATISTICS")
print(f"   Vertices: {G.number_of_vertices()}")
print(f"   Edges: {G.number_of_edges()}")
print(f"   Layers: {G.layers.count()}")
print(f"   Snapshots: {len(G._snapshots)}")

print("\n✅ TESTED FEATURES")
features = [
    "AnnNet Properties (X, obs, var, uns)",
    "LayerManager (add, remove, union, intersect, stats)",
    "IndexManager (entity/edge lookups)",
    "CacheManager (CSR/CSC caching)",
    "GraphView (filtering, predicates, materialization)",
    "Snapshot & Diff (versioning, comparison)",
    "I/O (save/load .annnet format)",
    "Cross-layer analytics",
    "Polars integration",
    "Performance benchmarks"
]

for i, feature in enumerate(features, 1):
    print(f"   {i:2d}. {feature}")

print("\n📈 LAYER DETAILS")
print(G.layers.summary())

print("\n🎯 RECOMMENDATIONS")
print("   1. Use views for large subgraph operations (lazy, efficient)")
print("   2. Create snapshots before major graph modifications")
print("   3. Use layers for temporal/contextual data organization")
print("   4. Leverage Polars for fast attribute queries")
print("   5. Cache CSR/CSC for repeated matrix operations")

print("\n" + "=" * 60)
print("✓ ALL TESTS COMPLETED SUCCESSFULLY!")
print("=" * 60)


ANNNET COMPLETE TEST SUMMARY

📊 GRAPH STATISTICS
   Vertices: 8
   Edges: 14
   Layers: 5
   Snapshots: 102

✅ TESTED FEATURES
    1. AnnNet Properties (X, obs, var, uns)
    2. LayerManager (add, remove, union, intersect, stats)
    3. IndexManager (entity/edge lookups)
    4. CacheManager (CSR/CSC caching)
    5. GraphView (filtering, predicates, materialization)
    6. Snapshot & Diff (versioning, comparison)
    7. I/O (save/load .annnet format)
    8. Cross-layer analytics
    9. Polars integration
   10. Performance benchmarks

📈 LAYER DETAILS
Layers: 5
├─ default: 0 vertices, 0 edges
├─ 2022: 4 vertices, 4 edges
├─ 2023: 5 vertices, 5 edges
├─ 2024: 7 vertices, 5 edges
└─ all_years: 5 vertices, 12 edges

🎯 RECOMMENDATIONS
   1. Use views for large subgraph operations (lazy, efficient)
   2. Create snapshots before major graph modifications
   3. Use layers for temporal/contextual data organization
   4. Leverage Polars for fast attribute queries
   5. Cache CSR/CSC for repeated

In [64]:
print("\n" + "=" * 60)
print("CLEANUP")
print("=" * 60)

import os
import shutil

# Clean up test files
files_to_remove = [
    "company_network.annnet",
    "network_2023.annnet",
]

print("\nRemoving test files:")
for filepath in files_to_remove:
    if os.path.exists(filepath):
        if os.path.isdir(filepath):
            shutil.rmtree(filepath)
            print(f"   ✓ Removed directory: {filepath}")
        else:
            os.remove(filepath)
            print(f"   ✓ Removed file: {filepath}")
    else:
        print(f"   ⊘ Not found: {filepath}")

print("\n✓ Cleanup complete")


CLEANUP

Removing test files:
   ✓ Removed directory: company_network.annnet
   ✓ Removed directory: network_2023.annnet

✓ Cleanup complete
